In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
lendingDF = pd.read_csv('Resources/lending_data.csv')

In [3]:
# Review the DataFrame
lendingDF.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [4]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = lendingDF['loan_status']

In [5]:
# Separate the X variable, the features
x = lendingDF[['loan_size', 'interest_rate', 'borrower_income', 'debt_to_income', 'num_of_accounts', 'derogatory_marks', 'total_debt']]

In [6]:
# Review the y variable Series
y[:5]

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [7]:
# Review the X variable DataFrame
x[:5]

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [8]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [9]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

In [10]:
# Split the data using train_test_split
# Assign a random_state of 1 to the function
x_train, x_test, y_train,y_test = train_test_split(x,y, random_state=1, stratify=y)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [11]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

In [12]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifierModel = LogisticRegression(solver='lbfgs', random_state=1)

In [13]:
# Fit the model using training data
classifierModel.fit(x_train, y_train)

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [14]:
# Make a prediction using the testing data
predictions = classifierModel.predict(x_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [15]:
# Print the balanced_accuracy score of the model
balanced_accuracy_score(y_test, predictions)

0.9442676901753825

In [16]:
# Generate a confusion matrix for the model
confusion_matrix(y_test, predictions)

array([[18679,    80],
       [   67,   558]])

In [17]:
# Print the classification report for the model
print(classification_report(y_test, predictions,target_names = ["healthy loan", "high-risk loan"]))

                precision    recall  f1-score   support

  healthy loan       1.00      1.00      1.00     18759
high-risk loan       0.87      0.89      0.88       625

      accuracy                           0.99     19384
     macro avg       0.94      0.94      0.94     19384
  weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The model's **accuracy** reveals its ability to make correct predictions, with 18,679 true positives and 558 true negatives. **Precision** indicates that the model excels at predicting healthy loans (precision 1) but slightly less so for high-risk loans (precision 0.87). **Recall** shows that the model is adept at identifying actual positive cases, particularly for healthy loans (recall 1) compared to high-risk loans (recall 0.89). However, an imbalance in **support** is evident in the training data, with 18,759 healthy loans and only 625 high-risk loans.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [21]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

In [22]:
# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
ros_model = RandomOverSampler(random_state=1)

In [23]:
# Fit the original training data to the random_oversampler model
x_resampled, y_resampled = ros_model.fit_resample(x_train, y_train)

In [26]:
# Count the distinct values of the resampled labels data
y_resampled.value_counts()

0    56277
1    56277
Name: loan_status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [28]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifierModel = LogisticRegression(solver='lbfgs', random_state=1)

In [29]:
# Fit the model using the resampled training data
classifierModel.fit(x_resampled, y_resampled)

LogisticRegression(random_state=1)

In [30]:
# Make a prediction using the testing data
predictions = classifierModel.predict(x_resampled)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [31]:
# Print the balanced_accuracy score of the model 
balanced_accuracy_score(y_resampled, predictions)

0.994180571103648

In [33]:
# Generate a confusion matrix for the model
confusion_matrix(y_resampled, predictions)

array([[55945,   332],
       [  323, 55954]])

In [34]:
# Print the classification report for the model
print(classification_report(y_resampled, predictions,target_names = ["healthy loan", "high-risk loan"]))

                precision    recall  f1-score   support

  healthy loan       0.99      0.99      0.99     56277
high-risk loan       0.99      0.99      0.99     56277

      accuracy                           0.99    112554
     macro avg       0.99      0.99      0.99    112554
  weighted avg       0.99      0.99      0.99    112554



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The model exhibits high **accuracy**, correctly identifying 55,945 true positives and 55,954 true negatives. In terms of **precision**, it excels equally in predicting both healthy loans (precision 0.99) and high-risk loans (precision 0.99), signifying its reliability when classifying positive outcomes. Likewise, for **recall**, the model performs consistently well in identifying actual positive cases for both healthy loans (recall 0.99) and high-risk loans (recall 0.99). Moreover, the **support** column indicates a balanced dataset in the training data, with 56,277 instances of both healthy and high-risk loans.